# Tool calls in LLaMa 3.1

LLaMa 3.1 is distributed as an instruction-tuned model with 8B, 70B, and 405B parameter variants. As part of the release, Meta mentioned that

> These are multilingual and have a significantly longer context length of 128K, state-of-the-art tool use, and overall stronger reasoning capabilities.

Let's dig into how we can use these models with tools, and run an eval to see how they compare to gpt-4o on a benchmark.


## Setup

You can access LLaMa 3.1 models through inference services like [Together](https://www.together.ai/), which has generous rate limits and OpenAI protocol compatibility. We'll use Together, through the
[Braintrust proxy](https://www.braintrust.dev/docs/guides/proxy) to access LLaMa 3.1 and OpenAI models.

To get started, make sure you have a Braintrust account and an API key for [Together](https://www.together.ai) and [OpenAI](https://platform.openai.com/). Make sure to plug them into your Braintrust account's
[AI secrets](https://www.braintrust.dev/app/settings?subroute=secrets) configuration and acquire a [BRAINTRUST_API_KEY](https://www.braintrust.dev/app/settings?subroute=api-keys). Feel free to put your BRAINTRUST_API_KEY in a `.env.local` file next to this notebook, or just hardcode it into the code below.


In [1]:
import dotenv from "dotenv";
import * as fs from "fs";

if (fs.existsSync(".env.local")) {
  dotenv.config({ path: ".env.local", override: true });
}


In [2]:
import { OpenAI } from "openai";
import { wrapOpenAI } from "braintrust";

const client = wrapOpenAI(
  new OpenAI({
    apiKey: process.env.BRAINTRUST_API_KEY,
    baseURL: "https://api.braintrust.dev/v1/proxy",
    defaultHeaders: { "x-bt-use-cache": "never" },
  })
);

const LLAMA31_8B = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo";
const LLAMA31_70B = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo";
const LLAMA31_405B = "meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo";

const response = await client.chat.completions.create({
  model: LLAMA31_8B,
  messages: [
    {
      role: "user",
      content: "What is the weather in Tokyo?",
    },
  ],
  max_tokens: 1024,
  temperature: 0,
});

console.log(response.choices[0].message.content);


(node:36351) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


However, I'm a large language model, I don't have real-time access to current weather conditions. But I can suggest some ways for you to find out the current weather in Tokyo:

1. **Check online weather websites**: You can visit websites like AccuWeather, Weather.com, or the Japan Meteorological Agency (JMA) website to get the current weather conditions in Tokyo.
2. **Use a weather app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get the current weather conditions in Tokyo.
3. **Check social media**: You can also check social media platforms like Twitter or Facebook to see if there are any updates on the weather in Tokyo.

That being said, I can provide you with some general information about the climate in Tokyo. Tokyo has a humid subtropical climate, with four distinct seasons:

* **Spring (March to May)**: Mild temperatures, with average highs around 18°C (64°F) and lows around 10°C (50°F).
* **Summer (June to August)**: Hot and h

As expected, the model can't answer the question without access to some tools. Traditionally, LLaMa models haven't supported tool calling. Some inference providers have attempted to solve this with controlled generation or similar methods, although to limited success. However, the [llama-agentic-system](https://github.com/meta-llama/llama-agentic-system/blob/main/llama_agentic_system/system_prompt.py#L71) codebase alludes to a new approach to tool calls:

```text
Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{"example_name": "example_value"}</function>
```

Let's see if we can make this work with the commonly used weather tool definition.


In [4]:
const weatherTool = {
  name: "get_current_weather",
  description: "Get the current weather in a given location",
  parameters: {
    type: "object",
    properties: {
      location: {
        type: "string",
        description: "The city and state, e.g. San Francisco, CA",
      },
    },
    required: ["location"],
  },
};

const toolPrompt = `You have access to the following functions:

Use the function '${weatherTool.name}' to '${weatherTool.description}':
${JSON.stringify(weatherTool)}

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

`;

const response = await client.chat.completions.create({
  model: LLAMA31_8B,
  messages: [
    {
      role: "system",
      content: toolPrompt,
    },
    {
      role: "user",
      content: "What is the weather in Tokyo?",
    },
  ],
  max_tokens: 1024,
  temperature: 0,
});

console.log(response.choices[0].message.content);


<function=get_current_weather>{"location": "Tokyo, JP"}</function>


Wow cool! Looks like we can get the model to call the tool. Let's quickly write a parser that can extract the function call from the response.


In [64]:
function parseToolResponse(response: string) {
  const functionRegex = /<function=(\w+)>(.*?)(?:<\/function>|$)/;
  const match = response.match(functionRegex);

  if (match) {
    const [, functionName, argsString] = match;
    try {
      const args = JSON.parse(argsString);
      return {
        functionName,
        args,
      };
    } catch (error) {
      console.error("Error parsing function arguments:", error);
      return null;
    }
  }

  return null;
}

// const parsedResponse = parseToolResponse(response.choices[0].message.content);
// console.log(parsedResponse);

console.log(
  parseToolResponse(
    `<function=select_choice>{"reasons":"The submitted answer is a superset of the expert answer and is fully consistent with it. The submitted answer contains additional information that is not present in the expert answer, but the core information is the same. For example, the submitted answer 'The weather in Tokyo is extremely hot' is a superset of the expert answer 'The weather in Tokyo is hot' because it contains additional information about the temperature being extremely high. However, the submitted answer is fully consistent with the expert answer in terms of the core information, which is that the weather in Tokyo is hot. Therefore, the correct answer is (B) The submitted answer is a superset of the expert answer and is fully consistent with it.","choice":"B"}`
  )
);
console.log(
  parseToolResponse(
    `<function=select_choice>{"choice":"E","reasons":"The submitted answer is a superset of the expert answer and is fully consistent with it."}</function>`
  )
);


[
  `<function=select_choice>{"reasons":"The submitted answer is a superset of the expert answer and is fully consistent with it. The submitted answer contains additional information that is not present in the expert answer, but the core information is the same. For example, the submitted answer 'The weather in Tokyo is extremely hot' is a superset of the expert answer 'The weather in Tokyo is hot' because it contains additional information about the temperature being extremely high. However, the submitted answer is fully consistent with the expert answer in terms of the core information, which is that the weather in Tokyo is hot. Therefore, the correct answer is (B) The submitted answer is a superset of the expert answer and is fully consistent with it.","choice":"B"}`,
  'select_choice',
  `{"reasons":"The submitted answer is a superset of the expert answer and is fully consistent with it. The submitted answer contains additional information that is not present in the expert answer, 

## A real use case: LLM-as-a-Judge evaluators that make tool calls

At Braintrust, we maintain a suite of evaluator functions in the [Autoevals](https://github.com/braintrustdata/autoevals) library. Many of these evaluators, like `Factuality`, are "LLM-as-a-Judge"
evaluators that use a well-crafted prompt to an LLM to reason about the quality of a response. We are big fans of tool calling, and leverage it extensively in `autoevals` to make it easy and reliable
to parse the scores and reasoning they produce.

As we change autoevals, we run evals to make sure we improve performance and avoid regressing key scenarios. We'll run some of our autoeval evals as a way of assessing how well LLaMa 3.1 stacks up to gpt-4o.

Here is a quick example of the `Factuality` scorer, a popular LLM-as-a-Judge evaluator that uses the following prompt:

```ansi
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
[BEGIN DATA]
************
[Question]: {{{input}}}
************
[Expert]: {{{expected}}}
************
[Submission]: {{{output}}}
************
[END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
(A) The submitted answer is a subset of the expert answer and is fully consistent with it.
(B) The submitted answer is a superset of the expert answer and is fully consistent with it.
(C) The submitted answer contains all the same details as the expert answer.
(D) There is a disagreement between the submitted answer and the expert answer.
(E) The answers differ, but these differences don't matter from the perspective of factuality.
```


In [6]:
import { Factuality } from "autoevals";

console.log(
  await Factuality({
    input: "What is the weather in Tokyo?",
    output: "The weather in Tokyo is scorching.",
    expected: "The weather in Tokyo is extremely hot.",
  })
);


{
  name: 'Factuality',
  score: 1,
  metadata: {
    rationale: '1. The expert answer states that the weather in Tokyo is "extremely hot."\n' +
      '2. The submitted answer states that the weather in Tokyo is "scorching."\n' +
      '3. Both "extremely hot" and "scorching" convey the same factual content, indicating very high temperatures.\n' +
      '4. There is no additional information in either answer that would make one a subset or superset of the other.\n' +
      '5. Therefore, the submitted answer contains all the same details as the expert answer.',
    choice: 'C'
  }
}


Now let's reproduce this with LLaMa 3.1.


In [20]:
import { templates } from "autoevals";
import * as yaml from "js-yaml";
import mustache from "mustache";

const template = yaml.load(templates["factuality"]);

const selectTool = {
  name: "select_choice",
  description: "Call this function to select a choice.",
  parameters: {
    properties: {
      reasons: {
        description:
          "Write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset.",
        title: "Reasoning",
        type: "string",
      },
      choice: {
        description: "The choice",
        title: "Choice",
        type: "string",
        enum: Object.keys(template.choice_scores),
      },
    },
    required: ["reasons", "choice"],
    title: "CoTResponse",
    type: "object",
  },
};

async function LLaMaFactuality({
  model,
  input,
  output,
  expected,
}: {
  model: string;
  input: string;
  output: string;
  expected: string;
}) {
  const toolPrompt = `You have access to the following functions:

Use the function '${selectTool.name}' to '${selectTool.description}':
${JSON.stringify(selectTool)}

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

Here are a few examples:

`;

  const response = await client.chat.completions.create({
    model,
    messages: [
      {
        role: "system",
        content: toolPrompt,
      },
      {
        role: "user",
        content: mustache.render(template.prompt, {
          input,
          output,
          expected,
        }),
      },
    ],
    temperature: 0,
    max_tokens: 2048,
  });

  try {
    const parsed = parseToolResponse(response.choices[0].message.content);
    return {
      name: "Factuality",
      score: template.choice_scores[parsed?.args.choice],
      metadata: {
        rationale: parsed?.args.reasons,
        choice: parsed?.args.choice,
      },
    };
  } catch (e) {
    return {
      name: "Factuality",
      score: null,
      metadata: {
        error: `${e}`,
      },
    };
  }
}

console.log(
  await LLaMaFactuality({
    model: LLAMA31_8B,
    input: "What is the weather in Tokyo?",
    output: "The weather in Tokyo is scorching.",
    expected: "The weather in Tokyo is extremely hot.",
  })
);


{
  name: 'Factuality',
  score: 0.6,
  metadata: {
    rationale: "The submitted answer 'The weather in Tokyo is scorching' is a superset of the expert answer 'The weather in Tokyo is extremely hot' because it includes the same information and adds more detail. The word 'scorching' is a synonym for 'extremely hot', so the submitted answer is fully consistent with the expert answer.",
    choice: 'B'
  }
}


Ok interesting! It parses but the response is a little different from the GPT-4o response. Let's put this to the test at scale with some evals.


## Running evals

We use a subset of the [CoQA](https://stanfordnlp.github.io/coqa/) dataset to test the Factuality scorer. Let's load the dataset and take a look at an example.


In [17]:
interface CoqaCase {
  input: {
    input: string;
    output: string;
    expected: string;
  };
  expected: number;
}

const data: CoqaCase[] = JSON.parse(
  fs.readFileSync("coqa-factuality.json", "utf-8")
);

console.log("Factuality");
console.log(await Factuality(data[1].input));

console.log("LLaMa-3.1-8B Factuality");
console.log(
  await LLaMaFactuality({
    model: LLAMA31_8B,
    ...data[1].input,
  })
);


Factuality
{
  name: 'Factuality',
  score: 0,
  metadata: {
    rationale: '1. The question asks about the color of Cotton.\n' +
      "2. The expert answer is 'white,' which directly addresses the color of Cotton.\n" +
      "3. The submitted answer is 'in a barn,' which does not address the color of Cotton at all.\n" +
      '4. Since the submitted answer does not provide any information about the color of Cotton, it conflicts with the expert answer.\n' +
      '\n' +
      'Therefore, there is a disagreement between the submitted answer and the expert answer.',
    choice: 'D'
  }
}
LLaMa-3.1-8B Factuality
{
  name: 'Factuality',
  score: 0,
  metadata: {
    rationale: "The submitted answer 'in a barn' does not contain any information about the color of Cotton, which is the topic of the question. The expert answer 'white' is a specific fact about Cotton. Therefore, the submitted answer is not a subset or superset of the expert answer, and it does not contain all the same details a

Not bad!

### GPT-4o

Let's run a full eval with gpt-4o, LLaMa-3.1-8B, LLaMa-3.1-70B, and LLaMa-3.1-405B to see how they stack up. Since the evaluator generates a number
between 0 and 1, we'll use the `NumericDiff` scorer to assess accuracy, and a custom `NonNull` scorer to measure how many invalid tool calls are generated.


In [18]:
import { Eval } from "braintrust";
import { NumericDiff } from "autoevals";

function NonNull({ output }: { output: number | null }) {
  return output !== null && output !== undefined ? 1 : 0;
}

const evalResult = await Eval("LLaMa-3.1-Tools", {
  data: data,
  task: async (input) =>
    (
      await Factuality({
        ...input,
        openAiDefaultHeaders: { "x-bt-use-cache": "never" },
      })
    ).score,
  scores: [NumericDiff, NonNull],
  experimentName: "gpt-4o",
  metadata: {
    model: "gpt-4o",
  },
});


RUNNING POST WITH URL https://www.braintrust.dev/api/experiment/register
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://www.braintrust.dev/api/base_experiment/get_id
RUNNING GET WITH URL https://d3p9hsahjc1cvu.cloudfront.net/experiment-comparison2


 ████████████████████████░░░░░░░░░░░░░░░░ | LLaMa-3.1-Tools [experimentName=gpt-4o]  |  60% | 60/100 datapoints



=========================SUMMARY=========================
gpt-4o-81ec7a1c compared to gpt-4o-310fd3e3:
84.58% (-1.67%) 'NumericDiff' score	(0 improvements, 1 regressions)
100.00% (0.00%) 'NonNull'     score	(0 improvements, 0 regressions)

4.28s 'duration'      	(13 improvements, 47 regressions)
0.00$ 'estimated_cost'	(20 improvements, 15 regressions)

See results for gpt-4o-81ec7a1c at https://www.braintrust.dev/app/braintrustdata.com/p/LLaMa-3.1-Tools/experiments/gpt-4o-81ec7a1c


It looks like GPT-4o does pretty well. Tool calling has been a highlight of OpenAI's feature set for a while, so it's not surprising that it's able to successfully parse 100% of the tool calls.

![gpt-4o-result](./assets/gpt-4o-result.png)

### LLama-3.1-8B, 70B, and 405B

Now let's evaluate each of the LLaMa-3.1 models.


In [21]:
for (const model of [LLAMA31_8B, LLAMA31_70B, LLAMA31_405B]) {
  await Eval("LLaMa-3.1-Tools", {
    data: data,
    task: async (input) => (await LLaMaFactuality({ model, ...input }))?.score,
    scores: [NumericDiff, NonNull],
    experimentName: model,
    metadata: {
      model,
    },
  });
}


RUNNING POST WITH URL https://www.braintrust.dev/api/experiment/register
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3


Error parsing function arguments: SyntaxError: Bad escaped character in JSON at position 61 (line 1 column 62)
    at JSON.parse (<anonymous>)
    at Proxy.parseToolResponse (evalmachine.<anonymous>:9:31)
    at Proxy.LLaMaFactuality (evalmachine.<anonymous>:98:32)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Object.task (evalmachine.<anonymous>:5:33)
    at async rootSpan.traced.name (/Users/ankur/projects/braintrust/sdk/js/dist/index.js:4552:26)
    at async callback (/Users/ankur/projects/braintrust/sdk/js/dist/index.js:4548:11)
    at async /Users/ankur/projects/braintrust/sdk/js/dist/index.js:4683:16


RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING PO

 ████████████████████████░░░░░░░░░░░░░░░░ | LLaMa-3.1-Tools [experimentName=meta-... |  60% | 60/100 datapoints
Evaluator LLaMa-3.1-Tools [experimentName=meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo] failed with 2 errors. This evaluation ("LLaMa-3.1-Tools [experimentName=meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo]") will not be fully logged.
InternalServerError: 520 <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.together.xyz | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" con


=========================SUMMARY=========================
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-571db11f compared to meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo-49a4defb:
74.41% 'NumericDiff' score	(0 improvements, 0 regressions)
68.97% 'NonNull'     score	(0 improvements, 0 regressions)

8.31s 'duration'      	(21 improvements, 39 regressions)
0.00$ 'estimated_cost'	(0 improvements, 0 regressions)

See results for meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-571db11f at https://www.braintrust.dev/app/braintrustdata.com/p/LLaMa-3.1-Tools/experiments/meta-llama%2FMeta-Llama-3.1-8B-Instruct-Turbo-571db11f


...

RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3


Error parsing function arguments: SyntaxError: Expected double-quoted property name in JSON at position 36 (line 1 column 37)
    at JSON.parse (<anonymous>)
    at Proxy.parseToolResponse (evalmachine.<anonymous>:9:31)
    at Proxy.LLaMaFactuality (evalmachine.<anonymous>:98:32)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Object.task (evalmachine.<anonymous>:5:33)
    at async rootSpan.traced.name (/Users/ankur/projects/braintrust/sdk/js/dist/index.js:4552:26)
    at async callback (/Users/ankur/projects/braintrust/sdk/js/dist/index.js:4548:11)
    at async /Users/ankur/projects/braintrust/sdk/js/dist/index.js:4683:16


RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://www.braintrust.dev/api/base_experiment/get_id
RUNNING GET WITH URL https://d3p9hsahjc1cvu.cloudfront.net/experiment-comparison2


 ████████████████████████░░░░░░░░░░░░░░░░ | LLaMa-3.1-Tools [experimentName=meta-... |  60% | 60/100 datapoints



=========================SUMMARY=========================
meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo-b2e826c2 compared to meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo-0f22d98e:
87.87% (+5.18%) 'NumericDiff' score	(6 improvements, 2 regressions)
88.33% (+1.67%) 'NonNull'     score	(6 improvements, 5 regressions)

7.73s 'duration'      	(20 improvements, 40 regressions)
0.00$ 'estimated_cost'	(0 improvements, 60 regressions)

See results for meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo-b2e826c2 at https://www.braintrust.dev/app/braintrustdata.com/p/LLaMa-3.1-Tools/experiments/meta-llama%2FMeta-Llama-3.1-405B-Instruct-Turbo-b2e826c2


In [24]:
// Pick 10 random examples from data
const examples = [...data];
examples.sort(() => Math.random() - 0.5);
console.log(examples.slice(0, 5));


[
  {
    input: {
      input: 'Who are the two boxer featured in this article?',
      output: 'Floyd Mayweather and Manny Pacquiao',
      expected: 'Floyd Mayweather and Manny Pacquiao'
    },
    expected: 1,
    metadata: {
      source: 'cnn',
      story: "(CNN)A chiseled boxer's Instagram feed shows him making constant references to the Bible and enjoying gospel singing with his wife. \n" +
        '\n' +
        'Another features his formidable opponent counting stacks of money, hanging out in strip clubs, and flashing diamond watches and Ferraris. \n' +
        '\n' +
        'Welcome to the world of boxing promotion, circa 2015. \n' +
        '\n' +
        'American Floyd Mayweather and Filipino Manny Pacquiao are set to officially announce their heavily anticipated boxing match at a press conference in Los Angeles Wednesday. \n' +
        '\n' +
        'With the combined purse for the May 2 bout in Las Vegas reported to touch $300 million pending viewership numbers, the 

In [57]:
// NOTE: We don't have expected rationales
const invertedChoiceScores = Object.fromEntries(
  Object.entries(template.choice_scores).map(([choice, score]) => [
    score,
    choice,
  ])
);

function formatExample(example: CoqaCase) {
  return `Input: ${JSON.stringify(example.input)}
Output: ${JSON.stringify({ choice: invertedChoiceScores[example.expected], reasons: example.expected === 1 ? "The answer is correct" : example.expected === 0 ? "The answer is irrelevant and non overlapping" : "The answer is a superset of the expected value" })}    
`;
}

formatExample(examples[0]);


Input: {"input":"Who are the two boxer featured in this article?","output":"Floyd Mayweather and Manny Pacquiao","expected":"Floyd Mayweather and Manny Pacquiao"}
Output: {"choice":"E","reasons":"The answer is correct"}    



In [59]:
async function LLaMaFactuality({
  model,
  input,
  output,
  expected,
}: {
  model: string;
  input: string;
  output: string;
  expected: string;
}) {
  const toolPrompt = `You have access to the following functions:

Use the function '${selectTool.name}' to '${selectTool.description}':
${JSON.stringify(selectTool)}

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
  `;

  const response = await client.chat.completions.create({
    model,
    messages: [
      {
        role: "system",
        content: toolPrompt,
      },
      {
        role: "user",
        content: mustache.render(template.prompt, {
          input,
          output,
          expected,
        }),
      },
      {
        role: "user",
        content: `Here are a few examples:

${examples.map(formatExample).join("\n")}`,
      },
    ],
    temperature: 0,
    max_tokens: 2048,
  });

  try {
    console.log(response.choices[0].message.content);
    const parsed = parseToolResponse(response.choices[0].message.content);
    return {
      name: "Factuality",
      score: template.choice_scores[parsed?.args.choice],
      metadata: {
        rationale: parsed?.args.reasons,
        choice: parsed?.args.choice,
      },
    };
  } catch (e) {
    console.error(e);
    return {
      name: "Factuality",
      score: null,
      metadata: {
        error: `${e}`,
      },
    };
  }
}

console.log(
  await LLaMaFactuality({
    model: LLAMA31_8B,
    input: "What is the weather in Tokyo?",
    output: "The weather in Tokyo is scorching.",
    expected: "The weather in Tokyo is extremely hot.",
  })
);


<function=select_choice>{"choice":"E","reasons":"The submitted answer is a superset of the expert answer and is fully consistent with it."}</function>


Error parsing function arguments: SyntaxError: Unexpected non-whitespace character after JSON at position 115 (line 1 column 116)
    at JSON.parse (<anonymous>)
    at Proxy.parseToolResponse (evalmachine.<anonymous>:9:31)
    at LLaMaFactuality (evalmachine.<anonymous>:47:32)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async evalmachine.<anonymous>:69:34
    at async Object.execute (/Users/ankur/projects/braintrust/cookbook/content/node_modules/.pnpm/tslab@1.0.21/node_modules/tslab/dist/executor.js:173:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/Users/ankur/projects/braintrust/cookbook/content/node_modules/.pnpm/tslab@1.0.21/node_modules/tslab/dist/jupyter.js:223:18)
    at async JupyterHandlerImpl.handleExecute (/Users/ankur/projects/braintrust/cookbook/content/node_modules/.pnpm/tslab@1.0.21/node_modules/tslab/dist/jupyter.js:181:21)
    at async ZmqServer.handleExecute (/Users/ankur/projects/braintrust/cookbook/content/node_

{
  name: 'Factuality',
  score: undefined,
  metadata: { rationale: undefined, choice: undefined }
}


In [56]:
const selectTool = {
  name: "select_choice",
  description: "Call this function to select a choice.",
  parameters: {
    properties: {
      reasons: {
        description:
          "Write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset.",
        title: "Reasoning",
        type: "string",
      },
      choice: {
        description: "The choice",
        title: "Choice",
        type: "string",
        enum: Object.keys(template.choice_scores),
      },
    },
    required: ["reasons", "choice"],
    title: "CoTResponse",
    type: "object",
  },
};

for (const model of [LLAMA31_8B /*, LLAMA31_70B, LLAMA31_405B*/]) {
  await Eval("LLaMa-3.1-Tools", {
    data: data,
    task: async (input) => (await LLaMaFactuality({ model, ...input }))?.score,
    scores: [NumericDiff, NonNull],
    experimentName: `${model}-few-shot`,
    metadata: {
      model,
    },
  });
}


RUNNING POST WITH URL https://www.braintrust.dev/api/experiment/register
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://d3p9hsahjc1cvu.cloudfront.net/logs3
RUNNING POST WITH URL https://www.braintrust.dev/api/base_experiment/get_id
RUNNING GET WITH URL https://d3p9hsahjc1cvu.cloudfront.net/experiment-comparison2


 ████████████████████████░░░░░░░░░░░░░░░░ | LLaMa-3.1-Tools [experimentName=meta-... |  60% | 60/100 datapoints



=========================SUMMARY=========================
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-few-shot-no-cot compared to meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-few-shot:
65.36% (-7.74%) 'NumericDiff' score	(8 improvements, 16 regressions)
100.00% (+21.67%) 'NonNull'     score	(13 improvements, 0 regressions)

3.20s 'duration'      	(32 improvements, 28 regressions)
0.00$ 'estimated_cost'	(3 improvements, 0 regressions)

See results for meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-few-shot-no-cot at https://www.braintrust.dev/app/braintrustdata.com/p/LLaMa-3.1-Tools/experiments/meta-llama%2FMeta-Llama-3.1-8B-Instruct-Turbo-few-shot-no-cot


## Where to go from here

In just a few minutes, we've cracked the code on how to perform tool calls with LLaMa-3.1 models and run a benchmark to compare their performance to GPT-4o. In doing so, we've
found a few specific areas for improvement, e.g. parsing errors for tool calls, and a surprising outcome that LLaMa-3.1-70B is better than both LLaMa-3.1-405B and GPT-4o, yet a
fraction of the cost.

To explore this further, you could:

- Expand the benchmark to measure other kinds of evaluators.
- Try providing few-shot examples or fine-tuning the models to improve their performance.
- Play with other models, like GPT-4o-mini or Claude to see how they compare.

Happy evaluating!
